In [160]:
#import relevant libraries

import requests
import json
from pprint import pprint
import operator
import numpy

# My Team info
team_id = 3402291


### Overall Player Transfer Data

In [161]:
# Request Information on all players today (now?)
base_url = 'https://fantasy.premierleague.com/api/'
r_all_players_today = requests.get(base_url+'bootstrap-static/').json()
pprint(r_all_players_today, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 10607976}


In [162]:
# Total Active players in the game at this point in time (estimate)

boolean_list_of_all_events_status = list(map(operator.itemgetter('is_current'), r_all_players_today['events']))
current_event = [item for item, condition in zip(r_all_players_today['events'], boolean_list_of_all_events_status) if condition]
current_gameweek = current_event[0].get('id')
total_active_players_estimate = int(round(r_all_players_today['total_players']*(0.97)**current_gameweek,))

In [169]:
# Net transfers of players at this moment 

all_players_transfers_in = list(map(operator.itemgetter('transfers_in'), r_all_players_today['elements']))
all_players_transfers_out = list(map(operator.itemgetter('transfers_out'), r_all_players_today['elements']))
net_transfers_overall_today = numpy.array(all_players_transfers_in) - numpy.array(all_players_transfers_out)

In [172]:
# Price Change so far this event 

all_players_price_change_for_current_event = list(map(operator.itemgetter('cost_change_event'), r_all_players_today['elements']))

In [176]:
# Players prices today

all_players_prices_today = list(map(operator.itemgetter('now_cost'), r_all_players_today['elements']))

In [177]:
# Player currently flagged red

list_of_all_players_status = list(map(operator.itemgetter('status'), r_all_players_today['elements']))
boolean_list_of_players_injured = [True if x == 'i' else False for x in list_of_all_players_status]

In [182]:
# Create DataFrame from dictionary of all this transfer data

price_change_dict = {
    "Price Change this night?": 0, #tomorrow we define this as "Player prices today" - "Player prices yesterday"
    "Net Transfers In/Out since last price change": 0, #we +"Net Transfers In/Out since yesterday every day, unless price change occurs, then we reset to ="Net Transfers In/Out since yesterday
    "Net Transfers In/Out since yesterday": 0, #tomorrow we define this as "Net Transfers Overall In/Out today" - "Net Transfers Overall In/Out yesterday"
    "Price Change so far this event ": all_players_price_change_for_current_event,
    "Total Active players in the game at this point in time (estimate)": total_active_players_estimate,
    "Player currently flagged as injured": boolean_list_of_players_injured,
    "Player prices today" : all_players_prices_today,
    "Player prices yesterday" : 0, #tomorrow we define this as "Player prices today" of the day before
    "Net Transfers Overall In/Out today" : net_transfers_overall_today, #not to be used in modelling
    "Net Transfers Overall In/Out yesterday" : 0 #tomorrow we define this as "Net Transfers overall today" of the day before #not to be used in modelling
}

# Dictionary into DataFrame
price_change_dataframe_jan_28th = pandas.DataFrame(price_change_dict)
price_change_dataframe_jan_28th

# Player transfer data is updated at least every 40 minutes, and probably every 30 or even every 15 minutes
# This dataframe is a snapshot at 1145pm GMT Jan 28th

,Price Change this night?,Net Transfers In/Out since last price change,Net Transfers In/Out since yesterday,Price Change so far this event,Total Active players in the game at this point in time (estimate),Player currently flagged as injured,Player prices today,Player prices yesterday,Net Transfers In/Out today,Net Transfers In/Out yesterday
0,0,0,0,0,5595501,False,44,0,-52459,0
1,0,0,0,0,5595501,False,39,0,-7530,0
2,0,0,0,0,5595501,True,44,0,-6276,0
3,0,0,0,0,5595501,False,54,0,-7146,0
4,0,0,0,0,5595501,False,50,0,-1052953,0
...,...,...,...,...,...,...,...,...,...,...
782,0,0,0,0,5595501,True,40,0,830,0
783,0,0,0,0,5595501,False,45,0,29,0
784,0,0,0,0,5595501,False,45,0,173,0
785,0,0,0,0,5595501,False,45,0,8,0


### Personal Team Data

In [ ]:
# Request Information on my team

r_my_team = requests.get(base_url+'entry/{}/event/{}/picks/'.format(team_id,current_gameweek)).json()
my_players_ids = list(map(operator.itemgetter('element'), r_my_team['picks']))
my_players_ids

# Get list of all player IDs
total_list_of_players_ids = list(map(operator.itemgetter('id'), r_all_players_today['elements']))

# Makes a list of all players that is TRUE is player is in my team
boolean_list_of_players_in_my_team = [player in my_players_ids for player in total_list_of_players_ids]

# Use boolean list to exact all info about my players this week
my_players_total = [item for item, condition in zip(r_all_players_today['elements'], boolean_list_of_players_in_my_team) if condition]
my_players_total

# Print all my player's names
for players in my_players_total:
    print(players.get('web_name'))